In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import deepRD
import deepRD.tools.analysisTools as analysisTools

In [ ]:
def read_numpy_arrays_from_file(file_path):
    arrays = []
    with open(file_path, 'r') as file:
        contents = file.read()

    # Split the file content based on square brackets [ ]
    array_strings = contents.split('[')[1:]

    # Convert each array string back to a numpy array and store it
    for array_string in array_strings:
        array = np.fromstring(array_string.replace(']', ''), sep=' ')
        arrays.append(array)

    return arrays

In [ ]:
# Load parameters from parameters file
simID = '0005'
localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + simID
parameterDictionary = analysisTools.readParameters(localDataDirectory + "/parameters_" + simID)
# Parameters for loading continuous trajectories from files (from original simulation)
numSimulations = parameterDictionary['numSimulations']
D = parameterDictionary['D'] 
dt = parameterDictionary['dt']
stride = parameterDictionary['stride']
tfinal = parameterDictionary['tfinal']
kappa = parameterDictionary['kappa'] 
sigma = parameterDictionary['sigma']
R = parameterDictionary['R']
cR = parameterDictionary['cR']
equilibrationSteps = parameterDictionary['equilibrationSteps']

In [ ]:
# Read files
file_path = localDataDirectory + '/classicSmol_nsims' + str(numSimulations) + '.xyz'
distancesArrays = read_numpy_arrays_from_file(file_path)

In [ ]:
print(len(distancesArrays),cR)

In [ ]:
# Calculate average concentration with bootstrapping stdDevs
nbins = 30
#numSimulations = 1500
# Calculate values of middle points of histogram
hist, bins = np.histogram(distancesArrays[0], bins=nbins, range=(sigma,R))
rvals = np.zeros(len(bins)-1)
rvolumes = np.zeros(len(bins)-1)
binsize = bins[1]-bins[0]
for i in range(len(bins)-1):
    rvals[i] = (bins[i+1] + bins[i])/2.0
    rvolumes[i] = 4*np.pi*(bins[i+1]**3 - bins[i]**3)/3.0 # Approx alternative: 4*np.pi* rvals[i]**2*(bins[i+1]-bins[i])
# Calculate average concentration
concentrationHistogram = np.zeros(len(bins)-1)
stdDevs = np.zeros(len(bins)-1)
concHist = [None]*numSimulations
for i in range(numSimulations):
    hist, bins = np.histogram(distancesArrays[i], bins=nbins, range=(sigma,R))
    concHist[i] = hist/rvolumes
for j in range(len(concentrationHistogram)):
    concentrationHistogram[j] = np.mean(np.array(concHist)[:,j])
    stdDevs[j] = np.std(np.array(concHist)[:,j])

In [ ]:
fig, ax = plt.subplots()
bw=0.1
fsize = 15
# Calculate analytical solution
kappa = 10
#sigma = 1.0 
#R = 10.0
#cR = 1.0
#rvals = np.arange(sigma,R,0.1)
kS = 4*np.pi*sigma*D
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))


ax.plot(rvals, analyticCK,'-k', linewidth =1, label='benchmark')
ax.plot(rvals, concentrationHistogram,'--b', linewidth = 2, label='simulation (n='+ str(numSimulations)+')')
ax.fill_between(rvals, concentrationHistogram-stdDevs, concentrationHistogram+stdDevs, alpha=0.1, 
                edgecolor='#1B2ACC', facecolor='#089FFF', linewidth=2, antialiased=True)
#ax.plot(rvals, analyticCKRinf,'-r', lw =3, label='benchmark Rinf')

ax.set_xlim((sigma+binsize, R-binsize))
ax.set_ylim((0, None))
ax.set_xlabel('radial distance',fontsize=fsize)
ax.set_ylabel('Concentration',fontsize=fsize)
ax.tick_params(axis='x', labelsize=fsize)
ax.tick_params(axis='y', labelsize=fsize)



ax.legend(bbox_to_anchor=(0.6, 0., 0.4, 0.3), framealpha=1.0,fontsize=fsize)

plt.tight_layout()

## Plots for paper

In [ ]:
# Calculate average concentration
nSimsList = [100,500,5000]
concentrationHistograms=[None]*len(nSimsList)
stdDevs = [None]*len(nSimsList)
for k, nsims in enumerate(nSimsList):
    concentrationHistograms[k] = np.zeros(nbins)
    stdDevs[k] = np.zeros(nbins)
    concHist = [None]*nsims
    for i in range(nsims):
        hist, bins = np.histogram(distancesArrays[i], bins=nbins, range=(sigma,R))
        concHist[i] = hist/rvolumes
    for j in range(len(concentrationHistograms[k])):
        concentrationHistograms[k][j] = np.mean(np.array(concHist)[:,j])
        stdDevs[k][j] = np.std(np.array(concHist)[:,j])

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,4), sharey=True, squeeze= True)
bw=0.1
fsize = 15

#Analytical solution
kS = 4*np.pi*sigma*D
kappa=8.5
analyticCKRinf = cR*(1 - kappa*sigma/(kS + kappa)*(1/rvals))
analyticCK = cR*(1 - kappa*sigma/(kS + kappa*(1-sigma/R))*(1.0/rvals - 1.0/R))

for k, nsims in enumerate(nSimsList):
    ax[k].plot(rvals, analyticCK,'-k', linewidth =1, label='benchmark')
    ax[k].plot(rvals, concentrationHistograms[k],'--b', linewidth = 2, label='simulation (n='+ str(nsims) + ')')
    ax[k].fill_between(rvals, concentrationHistograms[k]-stdDevs[k], concentrationHistograms[k]+stdDevs[k], alpha=0.1, 
                    edgecolor='#1B2ACC', facecolor='#089FFF', linewidth=2, antialiased=True)
    #ax.plot(rvals, analyticCKRinf,'-r', lw =3, label='benchmark Rinf')

    ax[k].set_xlim((sigma+binsize, R-binsize))
    ax[k].set_ylim((0, 1.25))
    ax[k].set_ylim((0, 0.75))
    ax[k].set_xlabel('r',fontsize=fsize)
    ax[k].tick_params(axis='x', labelsize=fsize)
    ax[k].tick_params(axis='y', labelsize=fsize)
    ax[k].legend(bbox_to_anchor=(0.6, 0., 0.4, 0.3), framealpha=1.0,fontsize=fsize)


ax[0].set_ylabel('Concentration',fontsize=fsize)

plt.tight_layout()
plt.savefig('Smoluchowski_plots_cR=' + str(cR) + '.pdf')

## Old plots

In [ ]:
fig, ax = plt.subplots()
bw=0.1
concDensity = stats.kde.gaussian_kde(distancesArrays[0],bw)

distance = np.arange(sigma, R, 0.02)
#ax.plot(distance, concDensity(distance), '-k') # label='benchmark')
ax.plot(distance, concDensity(distance)/(4*np.pi*distance**2),'--k') # label='benchmark')

ax.set_xlim((sigma, R))
ax.set_ylim((0, None))
ax.set_xlabel('radial distance')
ax.set_ylabel('Concentration')

#ax.legend(bbox_to_anchor=(0.6, 0., 0.5, 1.0), framealpha=1.0)

plt.tight_layout()